<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Model-1" data-toc-modified-id="Model-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model 1</a></span></li><li><span><a href="#Combining-results-with-Shuyuan's-data" data-toc-modified-id="Combining-results-with-Shuyuan's-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Combining results with Shuyuan's data</a></span></li></ul></div>

# Load

In [ ]:
import pickle

import matplotlib.pyplot as plt

import pandas as pd

from itertools import product

import numpy as np


In [ ]:
filename="results/stats_model_v1"

In [ ]:
infile = open(filename,'rb')
stats = pickle.load(infile) # data saved from simulation
infile.close()

# Model 1

- Timer for S/G2
- Sizer for G1/S

Keys of the dict are the parameters of the model, ordered in the following fashion: $(\alpha, \beta_0, \varepsilon, \gamma, transition\_th = M_{G1/S})$. 

For this first run, we have saved: 
- average RB amount and RB concentration over a given cycle
- delta of RB amount of RB concentration over a given cycle

The dimensionless parameters that we can look at are: 
- $ \Pi_1 = \frac{\alpha M_{G1/S} }{\beta_0 + \gamma}$
- $\Pi_2 = \beta/\gamma$

In [ ]:
def Pi1(key): 
    alpha, beta0, epsilon, gamma, M_G1S = key
    return alpha*M_G1S/(beta0 + gamma)

In [ ]:
def Pi2(key):
    alpha, beta0, epsilon, gamma, M_G1S = key
    return beta0/gamma

In [ ]:
def Pi3(key):
    alpha, beta0, epsilon, gamma, M_G1S = key
    return alpha*M_G1S/(gamma)

In [ ]:
def Pi4(key):
    alpha, beta0, epsilon, gamma, M_G1S = key
    return alpha/(beta0+gamma)

---

In [ ]:
keys = list(stats.keys())

In [ ]:
pi1 = np.array([Pi1(key) for key in keys])
pi2 = np.array([Pi2(key) for key in keys])
pi3 = np.array([Pi3(key) for key in keys])
pi4 = np.array([Pi4(key) for key in keys])

In [ ]:
pairs = list(zip(pi1, pi2))

In [ ]:
unique_pairs = []
for p in pairs:
    if p not in unique_pairs:
        unique_pairs.append(p)

In [ ]:
len(unique_pairs)/len(pairs)

About 20 % are actually unique. 

---

Let us first scatter plot all of these metrics: 

In [ ]:
def plot_results(pi1, pi2, stats, shift=.1, min_length=5, ylims=None, xlims=None):

    fig, ax = plt.subplots(2, 2, figsize = (10, 10))
    
    keys = list(stats.keys())
    keys_stats = list(stats[keys[0]].keys())
    for i, j in product(range(2), repeat=2):
        k = i + 2*j

        colors = []
        for m in keys:
            crt_ = stats[m][keys_stats[k]]

            if len(crt_)<=min_length:
                colors.append(np.nan)

            else:
                if keys_stats[k].endswith("delta"):
                    val = crt_[-1]/stats[m][keys_stats[k].split("delta")[0]+"avg"][-1]
                    title = keys_stats[k] + "/" + keys_stats[k].split("delta")[0]+"avg"
                    log_ = False
                else:
                    val = crt_[-1]
                    title = keys_stats[k]
                    log_ = True
                colors.append(val)

        valid = ~np.isnan(np.array(colors))

        dot_cols = np.array(colors)[valid]
        
        if log_: 
            dot_cols = np.log(dot_cols)
            
        p = ax[i,j].scatter(
            pi1[valid]*(1+shift*np.random.randn(len(valid[valid]))), 
            pi2[valid]*(1+shift*np.random.randn(len(valid[valid]))), 
            s=2,
            c=dot_cols, 
            alpha=.1, 
            cmap='tab20'
        )
        ax[i,j].set_title(title)
        ax[i,j].set_xscale('log')
        ax[i,j].set_yscale('log')
        ax[i,j].set_xlabel("Pi1")
        ax[i,j].set_ylabel("Pi2")
        ax[i,j].set_ylim(ylims)
        ax[i,j].set_xlim(xlims)
        cbar = fig.colorbar(p, ax=ax[i,j])

In [ ]:
plot_results(pi1, pi2, stats)

In [ ]:
plot_results(pi3, pi2, stats)

In [ ]:
plot_results(pi4, pi2, stats)

# Combining results with Shuyuan's data

Shuyuan's data seems to suggest that: 
- RB_delta/RB_avg is between .5 and 1. (i.e. large variations!) -- therefore it seems that $\pi_2$ is about .1. 
- G1 length is about 15 hours
- mass at transition is about 1500 pixels... Not sure how to incorporate that into the model just yet.
- Total cell cycle duration is about 27-28 hours. 
- G2 length is about 12 hours. 

In the simple model, we have that $M  = M_0 \exp(\gamma t)$. During G2, we see that the cells approximately grow 1.5 times (and much less spread than during G1). Therefore, we can write: 
$$
M =  1.5M_0 = M_0 \exp(\gamma \tau_{G2})
$$

Therefore, we can say that, approximately, $\gamma \sim \log(1.5)/\tau_{G2} \sim .4/\tau_{G2}.$ Saying that $\tau_{G2} = 12\pm 2$ hours, we have:$\gamma \in [3\cdot 10^{-2}, 4\cdot 10^{-2}]$.

If $\pi_2 \sim .1$, then $\beta = 0.1\gamma \Rightarrow \in [3\cdot 10^{-3}, 4\cdot 10^{-3}]$. 

In that case, $\pi_1 \sim \frac{\alpha M}{\gamma}$ (as $\beta$ is much smaller than $\gamma$). 

We now need to find a good value for $\alpha M$. Untangling both will probably be hard, but estimating $M$ from data should be possible? We see that $\pi_4$ is better able to separate data according to concentration. 

$\pi_4 = \alpha/(\beta_0 + \gamma) \sim \alpha /\gamma$. 

In the model, the concentration is simply RB_amount/Mass. We can take a reference value for RB concentration to be 100nM. In the model, we use transition masses between .5 and 3. The results would not change (relatively speaking) if we multiply the mass by some constant. So let's assume that the mass is actually a factor of some standard mass. The value of $\pi_1$ that you end up choosing will directly depend on this value I think, so not sure this is the best way to go about things. 

Actually I don't think it's a good idea right now to fix $\alpha$. 

Summary: 
- $\gamma \in [3\cdot 10^{-2}, 4\cdot 10^{-2}]$
- $ \beta \in [3\cdot 10^{-3}, 4\cdot 10^{-3}]$
- $M_{G1/S} \sim 1.5 * M_b$
- $\alpha$ ? 